
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [1]:
# python imports
import re
import json
from os.path import join

# third party imports
import pandas
import escher
import cobra.test

# ecoli me
from ecolime.flat_files import *

from minime import *
from minime.util.building import build_reactions_from_genbank, add_transcription_reaction
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

In [41]:
me.reactions.translation_b0002.reaction

'20 generic_tRNA_tyr__L_c + 20 tyr__L_c + 63 generic_tRNA_gly_c + 91 ala__L_c + 63 gly_c + 44 asp__L_c + 29 pro__L_c + 46 generic_tRNA_arg__L_c + 69 generic_tRNA_val__L_c + 0.0100342633382281*mu + 0.00392339696524719 ribosome + 30 generic_tRNA_gln__L_c + 12 cys__L_c + 38 generic_tRNA_asn__L_c + 34 lys__L_c + 44 generic_tRNA_asp__L_c + 91 generic_tRNA_ala__L_c + 46 ile__L_c + 14 his__L_c + 2e-08 RNA_b0002 + 69 val__L_c + 34 generic_tRNA_lys__L_c + 3280 h2o_c + 4 trp__L_c + 14 generic_tRNA_his__L_c + 38 asn__L_c + 51 generic_tRNA_ser__L_c + 30 gln__L_c + 30 generic_tRNA_phe__L_c + 89 leu__L_c + 23 generic_tRNA_met__L_c + 46 generic_tRNA_ile__L_c + 53 generic_tRNA_glu__L_c + 23 met__L_c + 30 phe__L_c + 46 arg__L_c + 53 glu__L_c + 34 generic_tRNA_thr__L_c + 29 generic_tRNA_pro__L_c + 51 ser__L_c + 34 thr__L_c + 1640 atp_c + 12 generic_tRNA_cys__L_c + 89 generic_tRNA_leu__L_c + 1640 gtp_c + 4 generic_tRNA_trp__L_c --> 3280 pi_c + 89.103546 biomass + 1640 gdp_c + 3280 h_c + 1640 adp_c + prot

## Begin by loading metabolites

In [2]:
def fix_id(id_str):
    return id_str.replace("_DASH_", "__")

In [3]:
me = MEmodel("iJO1366-ME")
me.id = "iJO1366-ME"
me.compartments = {"p": "Periplasm", "e": "Extra-organism",
                   "c": "Cytosol"}
compartment_lookup = {v: k for k, v in me.compartments.items()}

In [4]:
met_info = pandas.read_csv(
    join(ecoli_files_dir, "metabolites.txt"),
    delimiter="\t", header=None, index_col=0,
    names=["id", "name", "formula", "compartment", "data_source"])

In [5]:
for met_id in met_info.index:
    fixed_id = fix_id(met_id)
    for compartment in met_info.compartment[met_id].split("AND"):
        compartment = compartment.strip()
        if compartment == "No_Compartment":
            print "Assigned %s to e" % met_id
            compartment = me.compartments["e"]
        metab = Metabolite(fixed_id + "_" + compartment_lookup[compartment])
        metab.name = met_info.name[met_id]
    
        metab.formula = met_info.formula[met_id]
        me.add_metabolites(metab)

Assigned tl to e
Assigned hemed to e
Assigned pqq to e
Assigned dpm to e
Assigned 23bpg to e
Assigned tqn to e


Add generic ions to simplify complexes

In [6]:
divalent_list = divalent_list
monovalent_list = monovalent_list
comp = "_c"  # compartment
# Divalent
div = cobra.Metabolite('generic_divalent' + comp)
div.name = 'Generic divalent ion'
me.add_metabolites(div)
for ion in divalent_list:
    rxn1 = cobra.Reaction(ion + comp + '_to_generic')
    ion_dict = {}
    met1 = me.metabolites.get_by_id(ion + comp)
    ion_dict[met1] = -1
    ion_dict[div] = 1
    rxn1.add_metabolites(ion_dict)
    me.add_reaction(rxn1)
# Monovalent
mono = cobra.Metabolite('generic_monovalent' + comp)
mono.name = 'Generic monovalent ion'
for ion in monovalent_list:
    rxn2 = cobra.Reaction(ion + comp + '_to_generic')
    ion_dict = {}
    met2 = me.metabolites.get_by_id(ion + comp)
    ion_dict[met2] = -1
    ion_dict[mono] = 1
    rxn2.add_metabolites(ion_dict)
    me.add_reaction(rxn2)

## Build Metabolic reactions

In [7]:
reaction_info = get_reaction_info_frame()
reaction_dict = get_reaction_matrix_dict()

In [8]:
for r_id in reaction_info.index:
    reaction = StoichiometricData(r_id, me)
    reaction._stoichiometry = {fix_id(k): v\
                               for k, v in reaction_dict[r_id].items()}
    reaction.lower_bound = \
        -1000. if reaction_info.is_reversible[r_id] else 0.
    reaction.upper_bound = 1000.

Also make a dummy reaction

In [9]:
dummy = StoichiometricData("dummy_reaction", me)
dummy.lower_bound = 0
dummy.upper_bound = 1000
dummy._stoichiometry = {}

Boundary Reactions

In [10]:
sources_sinks = pandas.read_csv(join(ecoli_files_dir, "reaction_matrix_sources_and_sinks.txt"), delimiter="\t",
                                header=None, names=["rxn_id", "met_id", "compartment", "stoic"], index_col=1)
sources_sinks.index = [fix_id(i) for i in sources_sinks.index]

source_amounts = pandas.read_csv(join(ecoli_files_dir, "exchange_bounds.txt"),
                                 delimiter="\t", index_col=0, names=["met_id", "amount"])
source_amounts.index = [fix_id(i) for i in source_amounts.index]


for met_id in sources_sinks.index:
    model_met_id = met_id + "_" + compartment_lookup[sources_sinks.compartment[met_id]]
    # EX_ or DM_ + met_id
    reaction_id = sources_sinks.rxn_id[met_id][:3] + model_met_id
    reaction = cobra.Reaction(reaction_id)
    me.add_reaction(reaction)
    reaction.add_metabolites({me.metabolites.get_by_id(model_met_id): -1})
    # set bounds on exchanges
    if reaction.id.startswith("EX_") and met_id in source_amounts.index:
        reaction.lower_bound = -source_amounts.amount[met_id]

Set bounds on boundary reactions

## Build ribosome and RNA Polymerase

In [11]:
ribosome_complex = ComplexData("ribosome", me)
ribosome_components = ribosome_complex.stoichiometry
for i in ["b3851", "b3854", "b3855"]:
    ribosome_components["RNA_" + i] = 1
# 30S Listed as [rpsA -rpsU], sra 
# [rplA-rplF],  rplI, [rplK-rplY],
# [rpmA-rpmJ]

for i in ["b0911", "b0169", "b3314", "b3296", "b3303", "b4200", "b3341",
          "b3306", "b3230", "b3321", "b3297", "b3342", "b3298", "b3307", 
          "b3165", "b2609", "b3311", "b4202", "b3316", "b0023", "b3065",
          "b1480"]:
    ribosome_components["protein_" + i] = 1
    
# 50S listed as [rplA-rplF],rplJ, rplI, rplK [rplM-rplY], [rpmA-rpmJ]    
for i in ["b3984", "b3317", "b3320", "b3319", "b3308", "b3305", 
          "b3958", "b4203", "b3983", "b3231", "b3310", "b3301",
          "b3313", "b3294", "b3304", "b2606", "b1716", "b3186",
          "b3315", "b3318", "b3309", "b2185", 
          "b3185", "b3637", "b3312", "b3302", "b3936", "b1089", 
          "b3636", "b3703", "b1717", "b3299"]:
    ribosome_components["protein_" + i] = 1
# [rplJ, 2(2[rplL7])]
ribosome_components["protein_" + "b3986"] = 4

ribosome_complex.create_complex_formation()

Created <TranslatedGene protein_b1089 at 0x7f6bc103c510> in <ComplexFormation formation_ribosome at 0x7f6bc1098d50>
Created <TranslatedGene protein_b3637 at 0x7f6bc0fcf890> in <ComplexFormation formation_ribosome at 0x7f6bc1098d50>
Created <TranslatedGene protein_b3636 at 0x7f6bc0fcf8d0> in <ComplexFormation formation_ribosome at 0x7f6bc1098d50>
Created <TranslatedGene protein_b3304 at 0x7f6bc0fcf910> in <ComplexFormation formation_ribosome at 0x7f6bc1098d50>
Created <TranslatedGene protein_b4200 at 0x7f6bc0fcf950> in <ComplexFormation formation_ribosome at 0x7f6bc1098d50>
Created <TranslatedGene protein_b1716 at 0x7f6bc0fcf990> in <ComplexFormation formation_ribosome at 0x7f6bc1098d50>
Created <TranslatedGene protein_b1717 at 0x7f6bc0fcf9d0> in <ComplexFormation formation_ribosome at 0x7f6bc1098d50>
Created <TranslatedGene protein_b0023 at 0x7f6bc0fcfa10> in <ComplexFormation formation_ribosome at 0x7f6bc1098d50>
Created <TranslatedGene protein_b3306 at 0x7f6bc0fcfa50> in <ComplexForm

In [12]:
RNAP_complex = ComplexData("RNA_Polymerase", me)
RNAP_components = RNAP_complex.stoichiometry
# Core RNA Polymerase Enzyme
for i in {"b3295" : "rpoA", "b3988" : "rpoC", "b3987" : "rpoB"}:
    if i == "b3295":
        RNAP_components["protein_" + i] = 2
    else:
        RNAP_components["protein_" + i] = 1

RNAP_complex.create_complex_formation()

Created <TranslatedGene protein_b3295 at 0x7f6bc103c2d0> in <ComplexFormation formation_RNA_Polymerase at 0x7f6bc0fb4fd0>
Created <TranslatedGene protein_b3988 at 0x7f6bc103c390> in <ComplexFormation formation_RNA_Polymerase at 0x7f6bc0fb4fd0>
Created <TranslatedGene protein_b3987 at 0x7f6bc103c3d0> in <ComplexFormation formation_RNA_Polymerase at 0x7f6bc0fb4fd0>


### Add Transcription/Translation from the genbank file

In [13]:
build_reactions_from_genbank(me, join(ecoli_files_dir, "NC_000913.2.gb"))

TODO deal with selenocystine


{'3892674,3893241': 'RNA_b3713',
 '3698585,3699857': 'RNA_b3539',
 '141430,141967': 'RNA_b0125',
 '4070697,4071594': 'RNA_b3882',
 '3790848,3791706': 'RNA_b3618',
 '3973168,3974299': 'RNA_b3791',
 '4623934,4625317': 'RNA_b4389',
 '155460,156201': 'RNA_b0140',
 '4243251,4244442': 'RNA_b4034',
 '846480,847227': 'RNA_b0811',
 '4258343,4258598': 'RNA_b4047',
 '2347956,2349033': 'RNA_b2239',
 '1002661,1003177': 'RNA_b0943',
 '4407297,4408029': 'RNA_b4180',
 '1647632,1647821': 'RNA_b1575',
 '4163450,4164308': 'RNA_b3967',
 '587204,590177': 'RNA_b0568',
 '1921089,1921338': 'RNA_b4432',
 '2541853,2542645': 'RNA_b2426',
 '1244901,1246599': 'RNA_b1197',
 '602638,603886': 'RNA_b0577',
 '1757326,1758547': 'RNA_b1680',
 '3409674,3410559': 'RNA_b3262',
 '732998,733325': 'RNA_b0702',
 '3026545,3027034': 'RNA_b2886',
 '4491397,4492429': 'RNA_b4267',
 '1904274,1905084': 'RNA_b1822',
 '3548101,3550495': 'RNA_b3417',
 '4005779,4006400': 'RNA_b3823',
 '1698980,1700153': 'RNA_b1622',
 '937216,938560': 'RNA

Add a dummy protein in as well

In [14]:
dna_sequence = "ATG" + "TTT" * 5 + "TAT"*5+ "ACG"*5 + "GAT" *5 + "AGT"*5 + "TAA"
add_transcription_reaction(me, "dummy", {"dummy"}, dna_sequence)
me.add_metabolites(TranslatedGene("protein_" + "dummy"))
translation = TranslationReaction("translation_" + "dummy")
me.add_reaction(translation)
translation.translation_data = TranslationData("dummy", me, "RNA_dummy", "protein_dummy")
translation.translation_data.compute_sequence_from_DNA(dna_sequence)
translation.update()

complex_data = ComplexData("CPLX_dummy", me)
complex_data.stoichiometry = {}
complex_data.stoichiometry["protein_" + "dummy"] = 1
complex_data.create_complex_formation()

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [15]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

In [16]:
me.tRNA_data.tRNA_b1230.synthetase

'Tyr_RS_dim'

### Add in complex Formation with modifications

In [17]:
# ME_complex_dict is a dict of {'complex_id': ['bnum(count)']}
ME_complex_dict = get_complex_to_bnum_dict()
# some entries in the file need to be renamed
renames = {"MnmE_": "b3706", "MnmG_": "b3741", "YheM_": "b3344", "YheL_": "b3343", "YheN_": "b3345"}
rna_components = {"b3123"}  # component id should have RNA_ instead of protein_

for cplx, value in ME_complex_dict.iteritems():
    complex_data = ComplexData(cplx, me)
    for gene in value:
        stoichiometry = gene[6]
        bnum = gene[0:5]
        comp_id = "RNA_" + bnum if bnum in rna_components \
            else "protein_" + renames.get(bnum, bnum)
        try:
            complex_data.stoichiometry[comp_id] = float(stoichiometry)
        except:
            complex_data.stoichiometry[comp_id] = float(1)

In [18]:
# {modified_complex_id: ['unmodified_complex_id', {component_id: stoic}]
modification_dict = get_protein_modification_dict(generic=True)
for mod_cplx_id, mod_complex_info in iteritems(modification_dict):
    unmod_cplx_id, mods = mod_complex_info
    unmod_cplx = me.complex_data.get_by_id(unmod_cplx_id)
    cplx = ComplexData(mod_cplx_id, me)
    cplx.stoichiometry = unmod_cplx.stoichiometry
    cplx.translocation = unmod_cplx.translocation
    cplx.chaperones = unmod_cplx.chaperones
    if len(set(mod_cplx_id.split("_mod_")[1:])) == len(mods):
        for mod_comp, mod_count in iteritems(mods):
            mod_id = "mod_" + mod_comp
            try:
                mod = me.modification_data.get_by_id(mod_id)
            except:
                mod = ModificationData(mod_id, me)
                mod.stoichiometry = {mod_comp: -1}
            cplx.modifications[mod_id] = -mod_count
    else:
        print "TODO:", mod_cplx_id

TODO: hRNAP_mod_1:generic_divalent_mod_2:generic_divalent
TODO: GltX_mono_mod_generic_divalent_mod_1:generic_divalent


Some modifications are enzyme-catalyzed

TODO: Fe-S enzyme-catalyzed modifications

In [19]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}

alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [20]:
# todo bmocogdp_c mods
mod_catalysts = {'CPLX0-1762':'G6712-MONOMER', # FE-S modification
                 'TMAOREDUCTI-CPLX':'EG12195-MONOMER',
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER',
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', 
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER',
                 'NAP-CPLX':'NAPD-MONOMER',
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'}

In [21]:
target_list = ['4fe4s_c', 'LI_c', '2fe2s_c', '3fe4s_c',
               'NiFeCoCN2_c']#, 'sheme_c']
              #'tl_c', '23bpg_c', 'acetyl_c', 'dpm_c', 'hemed_c

#target_list.extend(
#    ['EG50003-MONOMER_mod_pan4p', 'CPLX0-782_mod_2:4fe4s',
#     'EG50003-MONOMER_mod_pan4p_mod_oc', 'CPLX0-782_mod_1:2fe2s_mod_1:4fe4s'])

for met_id in target_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

unknown metabolite '4fe4s_c' created
unknown metabolite 'LI_c' created
unknown metabolite '2fe2s_c' created
unknown metabolite '3fe4s_c' created
unknown metabolite 'NiFeCoCN2_c' created


Build all complex formation reactions

TODO: This shouldn't be prining out stuff. Fix them.

In [22]:
list(me.modification_data.mod_NiFeCoCN2_c.get_complex_data())

[<ComplexData CPLX0-250_FORMATEDEHYDROGH-MONOMER_mod_8:4fe4s_mod_2:2fe2s_mod_bmocogdp_mod_NiFeCoCN2 at 0x7f6bbfdb5e50>]

In [23]:
for c in me.complex_data:
    if "mod_acetyl_c" in c.modifications:
        print c

<ComplexData rpL7/12_mod_1:acetyl at 0x7f6bbfb44f90>


In [24]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

Created <Metabolite pqq_c at 0x7f6bbf784910> in <ComplexFormation formation_GLUCDEHYDROG-MONOMER_mod_pqq at 0x7f6bbf7848d0>
Created <Metabolite hemed_c at 0x7f6bbf0f2910> in <ComplexFormation formation_CYT-D-UBIOX-CPLX_mod_pheme_mod_hemed at 0x7f6bbf0f2890>
Created <Metabolite acetyl_c at 0x7f6bbf0f2cd0> in <ComplexFormation formation_rpL7/12_mod_1:acetyl at 0x7f6bbf0f2c50>
Created <Metabolite EG50003-MONOMER_mod_pan4p at 0x7f6bc127d350> in <ComplexFormation formation_GCVMULTI-CPLX_mod_lipo at 0x7f6bbf117790>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_lipo at 0x7f6bbf117810> in <ComplexFormation formation_GCVMULTI-CPLX_mod_lipo at 0x7f6bbf117790>
Created <Metabolite 23bpg_c at 0x7f6bbe5f96d0> in <ComplexFormation formation_PHOSGLYCMUTASE_mod_23bpg at 0x7f6bbe5f9650>
Created <Metabolite dpm_c at 0x7f6bbf96ced0> in <ComplexFormation formation_OHMETHYLBILANESYN-MONOMER_mod_dpm at 0x7f6bbf96ce50>
Created <Metabolite tl_c at 0x7f6bbf93dcd0> in <ComplexFormation formation_S-ADENMETSYN

Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [25]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_modified_complex(generic=True)

Fixed _DASH:  D__LACtex
Fixed _DASH:  L__LACD2
Fixed _DASH:  L__LACD3
Fixed _DASH:  D__LACt2pp
Fixed _DASH:  L__LACtex
Fixed _DASH:  L__LACt2rpp


In [26]:
for reaction_data in me.stoichiometric_data:
    # Some reactions have no complex associated. Determine if they are
    # spontaneous or orphan reactions.
    try:
        complexes = rxnToModCplxDict[reaction_data.id]
    except KeyError:
        # These are orphans catalyzed by a dummy
        if reaction_data.id == "dummy_reaction" or \
                not reaction_info.is_spontaneous[reaction_data.id]:
            complexes = ["CPLX_dummy"]
        # These are truly spontaneous
        else:
            complexes = [None]
    for complex_id in complexes:
        complex_data = me.complex_data.get_by_id(complex_id) if complex_id else None
        if reaction_data.lower_bound < 0:
            r = MetabolicReaction(reaction_data.id + "_REV_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = True
            r.update(create_new=True)
        if reaction_data.upper_bound > 0:
            r = MetabolicReaction(reaction_data.id + "_FWD_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = False
            r.update(create_new=True)

Created <Metabolite CPLX0-7817_mod_Oxidized at 0x7f6bbf588690> in <MetabolicReaction RNDR1b3_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7f6bbf588650>
Created <Metabolite GLUTAREDOXIN-MONOMER_mod_Oxidized at 0x7f6bbf588710> in <MetabolicReaction RNDR1b1_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7f6bbf5886d0>
Created <Metabolite GRXC-MONOMER_mod_Oxidized at 0x7f6bbf588790> in <MetabolicReaction RNDR1b4_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7f6bbf588750>
Created <Metabolite GRXB-MONOMER_mod_Oxidized at 0x7f6bbf588910> in <MetabolicReaction RNDR1b2_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7f6bbf5888d0>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_3ha at 0x7f6bbf588f10> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7f6bbf588ed0>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_act at 0x7f6bbf588f50> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7f6bbf588ed0>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_ddca at 0x7f6bbf528110> in <Me

In [27]:
# This reaction is weird
me.reactions.get_by_id('CITLY-CPLX_2tpr3dpcoa_FWD_G6340-MONOMER').reaction

'2tpr3dpcoa_c + 4.27350427350427e-6*mu G6340-MONOMER --> '

In [28]:
me.reactions.charging_tRNA_b0202.reaction

'0.000116225011622501*mu + 4.5443979544398e-5 RNA_b0202 + 0.000116225011622501*mu + 4.5443979544398e-5 ile__L_c --> generic_tRNA_ile__L_c'

In [29]:
me.reactions.translation_b0005.reaction

'3 generic_tRNA_tyr__L_c + 2 generic_tRNA_asn__L_c + 11 generic_tRNA_gly_c + 7 ala__L_c + 11 gly_c + 2e-08 RNA_b0005 + 9 pro__L_c + 7 generic_tRNA_arg__L_c + 5 generic_tRNA_val__L_c + 4 generic_tRNA_gln__L_c + 0.00119921683798336*mu + 0.000468893783651493 ribosome + 7 lys__L_c + 5 generic_tRNA_asp__L_c + 7 generic_tRNA_ala__L_c + 3 ile__L_c + 13 his__L_c + 5 val__L_c + 7 generic_tRNA_lys__L_c + 3 generic_tRNA_ile__L_c + 6 trp__L_c + 196 atp_c + 13 generic_tRNA_his__L_c + 2 asn__L_c + 3 tyr__L_c + 3 generic_tRNA_ser__L_c + 4 gln__L_c + 3 generic_tRNA_met__L_c + 7 leu__L_c + 392 h2o_c + 2 generic_tRNA_glu__L_c + 3 met__L_c + 7 arg__L_c + 5 asp__L_c + 2 glu__L_c + generic_tRNA_thr__L_c + 9 generic_tRNA_pro__L_c + 3 ser__L_c + thr__L_c + 7 generic_tRNA_leu__L_c + 196 gtp_c + 6 generic_tRNA_trp__L_c --> 392 pi_c + protein_b0005 + 392 h_c + 196 adp_c + 11.362058 biomass + 196 gdp_c'

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [31]:
for r in me.reactions:
    if isinstance(r, tRNAChargingReaction):
        r.update()
for r in me.reactions:
    if isinstance(r, TranslationReaction):
        r.update()
    if isinstance(r, TranscriptionReaction):
        r.update()
    if isinstance(r, MetabolicReaction):
        r.update()

In [32]:
me.reactions.charging_tRNA_b2816

<tRNAChargingReaction charging_tRNA_b2816 at 0x7f6bbcb52990>

In [33]:
me.reactions.translation_b0005.reaction

'3 generic_tRNA_tyr__L_c + 2 generic_tRNA_asn__L_c + 11 generic_tRNA_gly_c + 7 ala__L_c + 11 gly_c + 2e-08 RNA_b0005 + 9 pro__L_c + 7 generic_tRNA_arg__L_c + 5 generic_tRNA_val__L_c + 4 generic_tRNA_gln__L_c + 0.00119921683798336*mu + 0.000468893783651493 ribosome + 7 lys__L_c + 5 generic_tRNA_asp__L_c + 7 generic_tRNA_ala__L_c + 3 ile__L_c + 13 his__L_c + 5 val__L_c + 7 generic_tRNA_lys__L_c + 3 generic_tRNA_ile__L_c + 6 trp__L_c + 196 atp_c + 13 generic_tRNA_his__L_c + 2 asn__L_c + 3 tyr__L_c + 3 generic_tRNA_ser__L_c + 4 gln__L_c + 3 generic_tRNA_met__L_c + 7 leu__L_c + 392 h2o_c + 2 generic_tRNA_glu__L_c + 3 met__L_c + 7 arg__L_c + 5 asp__L_c + 2 glu__L_c + generic_tRNA_thr__L_c + 9 generic_tRNA_pro__L_c + 3 ser__L_c + thr__L_c + 7 generic_tRNA_leu__L_c + 196 gtp_c + 6 generic_tRNA_trp__L_c --> 392 pi_c + protein_b0005 + 392 h_c + 196 adp_c + 11.362058 biomass + 196 gdp_c'

Remove unused protein and mRNA to make the model solve faster (TODO remove unused complexes too)

In [34]:
for c_d in me.complex_data:
    c = c_d.complex
    if len(c.reactions) == 1:
        list(c.reactions)[0].delete(remove_orphans=True)
for p in me.metabolites.query("protein"):
    if len(p._reaction) == 1:
        list(p._reaction)[0].delete(remove_orphans=True)
for m in me.metabolites.query("RNA"):
    if len(m._reaction) == 1:
        list(m._reaction)[0].delete(remove_orphans=True)

This gives the total number of genes included

In [35]:
len(me.reactions.query("transcription"))

4320

## Attempt to set keffs

In [36]:
divalent_list = divalent_list
monovalent_list = ['_mod_k','_mod_na1']
from pickle import load
with open("test_keffs.pickle", "rb") as infile:
    old_keffs = load(infile)
keffs = {}

for keff, value in old_keffs.items():
    for i in divalent_list: 
        keff = keff.replace(i, 'generic_divalent')
    for i in monovalent_list: 
        keff = keff.replace(i, '_mod_generic_monovalent')
    keffs[keff] = value
    
for r in me.reactions:
    if isinstance(r, MetabolicReaction) and r.complex_data is None:
        continue
    if isinstance(r, MetabolicReaction) and r.complex_data.id != "CPLX_dummy":
        met_rxn = r
        key = met_rxn.id.replace("-", "_DASH_").replace("__", "_DASH_").replace(":","_COLON_")
        #key = met_rxn.id
        key = "keff_" + key.replace("_FWD_", "_").replace("_REV_", "_")

        matches = [i for i in keffs if key in i]
        # get the direction
        if met_rxn.reverse:
            matches = [i for i in matches if i.endswith("_reverse_priming_keff")]
        else:
            matches = [i for i in matches if i.endswith("_forward_priming_keff")]
        if len(matches) == 1:
            met_rxn.keff = keffs[matches[0]]
        elif len(matches) > 0:
            if len(matches) == len([i for i in matches if key + "_mod_"]):
                met_rxn.keff = keffs[matches[0]]
            else:
                print key, len(matches)
        else:  # len(matches) == 0
            print "no keff found for", key

no keff found for keff_METAT_S_DASH_ADENMETSYN_DASH_CPLX_mod_4_COLON_generic_monovalent_mod_1_COLON_generic_divalent
no keff found for keff_GLUTRR_CPLX0_DASH_3741


In [ ]:
me.reactions.translation_b0038.reaction

## Solve

In [37]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [38]:
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMp']
for reaction in KO_list:
    a = me.reactions.query(reaction+'_')
    for rxn in a:
        rxn.upper_bound = 0
        rxn.lower_bound = 0

In [39]:
expressions = compile_expressions(me)

In [40]:
solve_at_growth_rate(me, 0.1, compiled_expressions=expressions)

<Solution 0.10 at 0x7f6bbc11c490>

In [ ]:
binary_search(me, min_mu=0, max_mu=1.0, mu_accuracy=1e-6,
              compiled_expressions=expressions)

In [ ]:
import escher
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = me.get_metabolic_flux()
view.display_in_notebook()

In [ ]:
me.compute_solution_error()

In [ ]:
def find_met_reactions(met_id):
    return [i for i in me.metabolites.get_by_id(met_id).reactions]
            #if not isinstance(i, (TranslationReaction, TranscriptionReaction))]

In [ ]:
met = ""
r = cobra.Reaction("DM_" + met)
me.add_reaction(r)
r.reaction = met + " --> "

In [ ]:
fva_results = fva(me, .5, find_met_reactions(met), compiled_expressions=expressions)
res = pandas.DataFrame.from_dict(fva_results).T
res